In [38]:
from pandas import DataFrame
import pandas as pd
from os import path
import os  
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/beaubranton/de-zoomcamp/data-engineering-zoomcamp-main/02-workflow-orchestration/mage-zoomcamp-copy/peppy-citron-411704-888e6b9e01ee.json"

bucket_name = 'mage_zoomcamp_beau_branton_bucket'
project_id = 'peppy-citron-411704'

table_name = "batting_box_scores_test"
root_path = f'{bucket_name}/d1_baseball_project/{table_name}'


def delete_gcs_folder_if_date_exists(bucket_name, date_list):
    """Checks if a folder with a specific date exists in the bucket and deletes if found."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs()
    for folder_date in date_list:
        for blob in bucket.list_blobs():
            if f"/date={folder_date}/" in blob.name:
                print(f"Deleted blob: {blob.name}")
                blob.delete()
    

# Call the function
folder_date = ["2024-03-30", "2024-03-31"]  # Replace with the date you're looking for
delete_gcs_folder_if_date_exists(bucket_name, folder_date)
   



Deleted blob: d1_baseball_project/batting_box_scores_test/date=2024-03-30/
Deleted blob: d1_baseball_project/batting_box_scores_test/date=2024-03-30/983ce5def3f442e38e083e384ad7935b-0.parquet
Deleted blob: d1_baseball_project/batting_box_scores_test/date=2024-03-31/
Deleted blob: d1_baseball_project/batting_box_scores_test/date=2024-03-31/49a7788ea8f54953a9849ba9a9d73e4b-0.parquet


In [4]:
from google.cloud import storage
import os
# from mage_ai.io.google_cloud_storage import GoogleCloudStorage
# from mage_ai.io.config import ConfigFileLoader
# from mage_ai.settings.repo import get_repo_path
from os import path
import pandas as pd
import urllib.parse
from datetime import datetime, timedelta

def extract_dates(object_keys):
    dates = []
    for key in object_keys:
        key = urllib.parse.unquote(key)
        parts = key.split('/')
        for part in parts:
            if part.startswith('date='):
                date = part[len('date='):].split('%')[0]
                # Convert the string to a datetime object and then to a date object
                date = datetime.strptime(date[:19], '%Y-%m-%d %H:%M:%S').date()
                dates.append(date)
    return dates

###SHOULD WE ALSO CHECK BIG QUERY TO SEE WHICH DATA ALREADY EXISTS?
### AND ONLY TRANSFER IN THE NEW DATA? OR SHOULD WE RUN THE WHOLE THING?

# config_path = path.join(get_repo_path(), 'io_config.yaml')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'temp.json'
config_profile = 'default'
bucket_name = 'mage_zoomcamp_beau_branton_bucket'
prefix = 'd1_baseball_project/batting_box_scores_test/date='

# Create a client
client = storage.Client()
# Get the bucket
bucket = client.get_bucket(bucket_name)
# Get the blobs (objects) with the specified prefix
blobs = bucket.list_blobs(prefix=prefix)
# Get the object keys
object_keys = [blob.name for blob in blobs if blob.name.endswith('.parquet')]

date_strings = extract_dates(object_keys)
dates = [date_string for date_string in date_strings]

#Test for checking date type of dates
# for i, element in enumerate(dates):
#     print(f"Element {i} is of type: {type(element)}")
# print(dates)

####START # Find the dates that are not in your dates list

start_date_str = '02/16/2024' #Replace with automatic variable for first day of season
end_date = datetime.now().date()

start_date = datetime.strptime(start_date_str, "%m/%d/%Y").date()
all_dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

for i, element in enumerate(all_dates):
    print(f"2nd: Element {i} is of type: {type(element)}")

missing_dates = [date for date in all_dates if date not in dates]
print(missing_dates)

####END # Find the dates that are not in your dates list

df = pd.DataFrame()

# Load each object
for object_key, date in zip(object_keys, dates):
    data = GoogleCloudStorage.with_config(ConfigFileLoader(config_path, config_profile)).load(
        bucket_name,
        object_key,
        format='parquet'
    )

    # ADD DATE BACK AS A COLUMN

    df = pd.concat([df,data])
    print(f"Finished appending data from {date}")


# Return the DataFrame
print(df)


DefaultCredentialsError: File temp.json was not found.